# Chemical-Disease Relation (CDR) Tutorial

In this example, we'll be writing an application to extract *mentions of* **chemical-induced-disease relationships** from Pubmed abstracts, as per the [BioCreative CDR Challenge](http://www.biocreative.org/resources/corpora/biocreative-v-cdr-corpus/).  At core, we will be constructing a model to classify _candidate_ CDR mentions as either true or false.

## Part IV: Training a Model with Data Programming

In this part of the tutorial, we will train a statistical model to differentiate between true and false `ChemicalDisease` mentions.

We will train this model using _data programming_, and we will **ignore** the training labels provided with the training data. This is a more realistic scenario; in the wild, hand-labeled training data is rare and expensive. Data programming enables us to train a model using only a modest amount of hand-labeled data for validation and testing. For more information on data programming, see the [NIPS 2016 paper](https://arxiv.org/abs/1605.07723).

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

# Note: We run automated tests on this tutorial to make sure that it is always up to date! 
# However, certain interactive components cannot currently be tested automatically, and will 
# be skipped with if-then statements using the variable below
AUTOMATED_TESTING = os.environ.get('TESTING') is not None

import numpy as np
from snorkel import SnorkelSession
session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


We repeat our definition of the `ChemicalDisease` `Candidate` subclass from Parts II and III.

In [2]:
from snorkel.models import candidate_subclass

ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])

## Loading `CandidateSet` objects

We reload the `CandidateSet` objects from the previous parts of the tutorial. Note that we will now process all three (training, validation, and test) as we go, because each plays a distinct role in Parts IV and V.

In [3]:
from snorkel.models import CandidateSet

train = session.query(CandidateSet).filter(CandidateSet.name == 'CDR Training Candidates').one()
dev = session.query(CandidateSet).filter(CandidateSet.name == 'CDR Development Candidates').one()
test = session.query(CandidateSet).filter(CandidateSet.name == 'CDR Test Candidates').one()

candidate_sets = [train, dev, test]

## Creating Labeling Functions
Labeling functions are a core tool of data programming. They are heuristic functions that aim to classify candidates correctly. Their outputs will be automatically combined and denoised to estimate the probabilities of training labels for the training data.

## Inspecting some examples in the training set

We'll start here to come up with some ideas for LFs:

In [4]:
from snorkel.viewer import SentenceNgramViewer

# Load the annotations
train_gold = session.query(CandidateSet).filter(CandidateSet.name == 'CDR Training Candidates -- Gold').one()

# NOTE: This if-then statement is only to avoid opening the viewer during automated testing of this notebook
# You should ignore this!
if not AUTOMATED_TESTING:
    sv = SentenceNgramViewer(train_gold, session, annotator_name="Tutorial Part IV User")
else:
    sv = None

<IPython.core.display.Javascript object>

In [5]:
sv

## Traditional "distant supervision" as a single LF

See http://ctdbase.org/downloads/;jsessionid=5B8E7F187A4772BB9478B6B3D9FCA5D1#cd

**TODO: Make download script**

In [6]:
DATA_ROOT = os.environ['SNORKELHOME'] + '/tutorial/data/'

CTD_lower = set()
with open(DATA_ROOT + 'dicts/CTD_chemicals_diseases.tsv', 'rb') as f:
    for line in f:
        if not line.startswith("#"):
            chem, chem_id, _, disease, disease_id, evidence, _, _, _, _ = line.split('\t')
            CTD_lower.add((chem.lower(), disease.lower()))
print len(CTD_lower)

1914160


In [7]:
from random import random

def LF_in_CDT_lower(c, p_neg=0.1):
    """Match against the CDT KB, with random negative supervision as well"""
    chem    = c.chemical.get_span().lower()
    disease = c.disease.get_span().lower()
    if (chem, disease) in CTD_lower:
        return 1
    else:
        return -1 if random() < p_neg else 0

In [8]:
from snorkel.lf_helpers import get_matches

matches = get_matches(LF_in_CDT_lower, train, match_values=[1])
SentenceNgramViewer(matches, session)

1134 matches


<IPython.core.display.Javascript object>

In [9]:
len(set(matches).intersection(train_gold.candidates))

379

In [10]:
len(train_gold)

1745

In [11]:
wrong = set(matches).difference(train_gold.candidates)
sv2 = SentenceNgramViewer(wrong, session)
sv2

<IPython.core.display.Javascript object>

In [ ]:
c = sv2.get_selected()
c

In [ ]:
[c for c in train_gold if c.chemical.parent.stable_id == '16867021::sentence:1536:1720']

In [ ]:
c.chemical.parent.stable_id

In [ ]:
tgc = set(train_gold.candidates.all())
len(tgc)

In [ ]:
len(set(matches).intersection(tgc))

In [ ]:
len(matches)

In [ ]:
len(train_gold)

### Using heuristic regex patterns as LFs

In [ ]:
sv.get_selected().get_arguments()

In [ ]:
LF_in_CDT(sv.get_selected())

## Applying Labeling Functions

First we construct a `CandidateLabeler`.

In [ ]:
from snorkel.annotations import LabelManager

label_manager = LabelManager()

Next we run the `CandidateLabeler` to to apply the labeling functions to the training `CandidateSet`.

In [ ]:
%time L = label_manager.create(session, train_1K, 'LF Labels', f=LFs)
L

**OR** load if we've already created:

In [ ]:
L = label_manager.load(session, train_1K, 'LF Labels')
L

We can also rerun a single labeling function (or more!) with the below command.

Do this to test changes to the labeling functions.

In [ ]:
def LF_test_3(candidate):
    return -1 if random.random() < 0.2 else 0

L = label_manager.update(session, train_1K, 'LF Labels', f=[LF_test_3])
L

We can view statistics about the resulting label matrix:

In [ ]:
L.lf_stats()

## Fitting the Generative Model
We estimate the accuracies of the labeling functions without supervision. Specifically, we estimate the parameters of a `NaiveBayes` generative model.

In [ ]:
from snorkel.learning import NaiveBayes

gen_model = NaiveBayes()
gen_model.train(L)

In [ ]:
gen_model.w

In [ ]:
gen_model.save(session, 'Generative Params 2')

In [ ]:
gen_model.load(session, 'Generative Params 2')
gen_model.w

We now apply the generative model to the training candidates.

In [ ]:
train_marginals = gen_model.marginals(L)

## Training the Discriminative Model
We use the estimated probabilites to train a discriminative model that classifies each `Candidate` as a true or false mention.

In [ ]:
from snorkel.learning import LogReg

disc_model = LogReg()
disc_model.train(F, train_marginals, n_iter=1500, rate=1e-5)

In [ ]:
disc_model.w.shape

In [ ]:
%time disc_model.save(session, "Discriminative Params 1")

In [ ]:
w_prev = disc_model.w
%time disc_model.load(session, "Discriminative Params 1")
np.all(disc_model.w == w_prev)

## Evaluating on the Development `CandidateSet`

In [ ]:
test_labels=[]
for candidate in sorted_test_candidates:
    test_labels.append(1 if candidate in gold_candidate_set else -1)
test_labels = np.asarray(test_labels)

score(sorted_test_candidates, test_labels, pred, gold_candidate_set, \
      train_marginals=train_marginals, test_marginals=test_marginals)

After evaluating on the development `CandidateSet`, the labeling functions can be modified. Try changing the labeling functions to improve performance. You can view the true positives, false positives, true negatives, and false negatives using the `Viewer`.

## Saving the Discriminative Model's Parameters
We save the model's parameters for use in Part V.

Next, in Part V, we will test our model on the test `CandidateSet`.